<a href="https://colab.research.google.com/github/jmmiddour/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-13 08:19:46--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.205.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.205.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’

instacart_online_gr 100%[===================>] 196.03M  67.1MB/s    in 2.9s    

2020-05-13 08:19:49 (67.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
# Import libraries I need to complete the assignment:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Read files
First need to read the files to find out which one(s) have the data I need.

In [0]:
# Read in the aisles file.
# Nothing here that I need.
pd.read_csv('aisles.csv').sample(5)

,aisle_id,aisle
71,72,condiments
109,110,pickled goods olives
39,40,dog food care
50,51,preserved dips spreads
26,27,beers coolers


In [0]:
# Read in the departments file. 
# Nothing here that I need.
pd.read_csv('departments.csv').sample(5)

,department_id,department
2,3,bakery
4,5,alcohol
19,20,deli
14,15,canned goods
1,2,other


In [0]:
# Read in the order products prior(test) file.
# This does have product id and reordered.
prior = pd.read_csv('order_products__prior.csv')
prior.sample(5)

,order_id,product_id,add_to_cart_order,reordered
12491483,1318404,23360,12,0
12089259,1276150,18337,6,0
30667313,3234704,41570,13,1
16121692,1700946,39657,3,0
2866078,302577,45578,7,1


In [0]:
# Read in the order products train file.
# This does have product id and reordered just like the prior file does. 
# Will need to concat this with the prior file.
train = pd.read_csv('order_products__train.csv')
train.sample(5)

,order_id,product_id,add_to_cart_order,reordered
1077522,2663702,34358,8,1
553758,1361650,32020,24,1
244718,596897,27783,13,0
443064,1084745,46979,6,1
1332763,3293410,6187,6,1


In [0]:
# Read in the orders file.
# Nothing here that I need.
pd.read_csv('orders.csv').sample(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1537764,959926,92369,prior,18,1,15,7.0
738174,3096888,44438,prior,2,1,12,25.0
670958,2607837,40449,prior,5,3,8,7.0
2494518,22382,150154,prior,6,5,8,6.0
3399099,741378,204846,prior,2,1,17,12.0


In [0]:
# Read in the products file.
# This does have product id and product name.
products = pd.read_csv('products.csv')
products.sample(5)

,product_id,product_name,aisle_id,department_id
3689,3690,Fruitables Apple Harvest Fruits & Vegetables,98,7
13382,13383,Jammy Sammy Apple Cinnamon & Oatmeal Snack Siz...,3,19
37259,37260,Dark Chocolate Cold Brew,100,21
46551,46552,"Nutrition Bar, Dark Chocolate Crunch",3,19
3809,3810,Eco-Foil Loaf Pans (8 in x 3 7/8 in x 2 15/32 in),10,17


#### Filter the Products
Need to filter the products file first to find out what the product ids are for the products I need.

From the product file I need to filter out:

- product_id
- product_name

In [0]:
# Filter out just the columns I need:
cols = ['product_id', 'product_name']
products = products[cols]
products.sample(5)

,product_id,product_name
22300,22301,Color Reflect Daily Moisture Conditioner
45615,45616,Organic French-Style Meyer Lemon Yogurt
9501,9502,Menthol Nasal Swab
29875,29876,Stax Cheddar Potato Crisps
46005,46006,Organic Whole Pineapple


In [0]:
# Create a list of the values I need:
prod_name = ['Banana', 'Bag of Organic Bananas', 'Organic Strawberries', 
             'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado',
             'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk', ]

In [0]:
# Subset the products dataframe to only the ones I need:
con = products['product_name'].isin(prod_name)

# Show the shape before applying the condition for a comparison:
print (products.shape)

# Apply the condition:
products = products[con]

# Show the new shape and dataframe to check my work:
print (products.shape)
products

(49688, 2)
(10, 2)


,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


#### Concatenate prior(test) and train
Need to concat the prior and train files for order products because I want all the row of both files in the same dataframe. They both have identical columns. Therefore, conatenating would be better then merging for this process.

In [0]:
# Check the shape on both files:
print (prior.shape)
print (train.shape)

(32434489, 4)
(1384617, 4)


In [0]:
# Concat them together:
order_products = pd.concat([prior, train], axis=0)
order_products.shape

(33819106, 4)

In [0]:
# Want to check my work by showing the first 5 of the new dataset:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


#### Filter the Order Products
Need to filter my new order products datasest to what I need.

From the concatinated order products file I need to filter out:
- product_id
- add_to_cart_order
- reordered

Also need to filter the rows based on the product_id shown for the products I need from my filtered products dataset.

In [0]:
# First need to filter out only the columns that I need:
cols2 = ['product_id', 'add_to_cart_order', 'reordered']
order_products = order_products[cols2]
order_products.head()

,product_id,add_to_cart_order,reordered
0,33120,1,1
1,28985,2,1
2,9327,3,0
3,45918,4,1
4,30035,5,0


In [0]:
# Create a list of only the values I need:
prod_id = [13176, 16797, 21137, 21903, 24852, 26209, 27845, 47209, 47626, 47766]

In [0]:
# Now need subset the new dataframe to only the rows I need:
con2 = order_products['product_id'].isin(prod_id)

# Show the shape before applying the condition for a comparison:
print (order_products.shape)

# Apply the condition:
order_products = order_products[con2]

# Show the new shape and dataframe to check my work:
print (order_products.shape)
order_products.sample(10)

(33819106, 3)
(2418314, 3)


,product_id,add_to_cart_order,reordered
23776459,13176,4,1
9401917,21903,3,1
24176193,21137,16,1
30678423,47209,5,0
7115703,47209,3,0
25797850,24852,17,1
50519,13176,4,1
3379957,24852,3,1
18218678,26209,2,1
3606088,24852,7,1


In [0]:
# Check my work to verify I only have the product ids that I need now:
order_products['product_id'].value_counts()

24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
47766    184224
47626    160792
16797    149445
26209    146660
27845    142813
Name: product_id, dtype: int64

#### Merge the Datasets
Need to merge my new filtered datasets together.

In [0]:
# Show the shape and first 3 rows of the new products dataset to find the 
#   common key:
print (products.shape)
products.head(3)

(10, 2)


,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries


In [0]:
# Show the shape and first 3 rows of the new order_products dataset to find the
#   common key:
print (order_products.shape)
order_products.head(3)

(2418314, 3)


,product_id,add_to_cart_order,reordered
12,21903,4,1
30,13176,1,1
51,47209,22,0


In [0]:
# Need to merge the datasets on the common key which is product_id:
products_ordered = pd.merge(products, order_products, on='product_id', how='outer')
products_ordered.head(10)

,product_id,product_name,add_to_cart_order,reordered
0,13176,Bag of Organic Bananas,1,1
1,13176,Bag of Organic Bananas,1,1
2,13176,Bag of Organic Bananas,3,1
3,13176,Bag of Organic Bananas,3,1
4,13176,Bag of Organic Bananas,3,1
5,13176,Bag of Organic Bananas,6,1
6,13176,Bag of Organic Bananas,4,1
7,13176,Bag of Organic Bananas,5,1
8,13176,Bag of Organic Bananas,1,1
9,13176,Bag of Organic Bananas,1,1


In [0]:
# I want to rename the columns to give it a cleaner look:
products_ordered.rename(columns={'product_id':'Product Id',
                                 'product_name':'Product Name',
                                 'add_to_cart_order':'Ordered',
                                 'reordered':'Reordered'},
                        inplace=True)
products_ordered.head() 

,Product Id,Product Name,Ordered,Reordered
0,13176,Bag of Organic Bananas,1,1
1,13176,Bag of Organic Bananas,1,1
2,13176,Bag of Organic Bananas,3,1
3,13176,Bag of Organic Bananas,3,1
4,13176,Bag of Organic Bananas,3,1


In [0]:
# Need to add the Ordered and Reordered columns together to get total ordered:
sum_col = products_ordered['Ordered'] + products_ordered['Reordered']
products_ordered['Total'] = sum_col
products_ordered.head()

,Product Id,Product Name,Ordered,Reordered,Total
0,13176,Bag of Organic Bananas,1,1,2
1,13176,Bag of Organic Bananas,1,1,2
2,13176,Bag of Organic Bananas,3,1,4
3,13176,Bag of Organic Bananas,3,1,4
4,13176,Bag of Organic Bananas,3,1,4


In [0]:
# Check for missing values in Product Name:
products_ordered['Product Name'].isnull().sum()

0

In [0]:
# Check for missing values in Total:
products_ordered['Total'].isnull().sum()

0

In [0]:
# Get the value counts of each product:
products_ordered['Product Name'].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: Product Name, dtype: int64

In [0]:
products_ordered['Total'].sum()

17263512

In [0]:
products_ordered['Product Name'].value_counts().sum()

2418314

In [0]:
products_ordered.head()

,Product Id,Product Name,Ordered,Reordered,Total
0,13176,Bag of Organic Bananas,1,1,2
1,13176,Bag of Organic Bananas,1,1,2
2,13176,Bag of Organic Bananas,3,1,4
3,13176,Bag of Organic Bananas,3,1,4
4,13176,Bag of Organic Bananas,3,1,4


In [0]:
# Create a visualization of my data using a bar plot:
products_ordered.plot('Product Name', 'Total', kind='barh', figsize=(16, 5))
plt.ylabel('Product Name', fontsize=14, fontweight='bold')

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy

In [0]:
# tidy -> table2

In [0]:
flights = sns.load_dataset('flights')

In [0]:
# Flights Pivot Table

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####